In [ ]:
import re

import numpy as np
from tensorflow.keras.layers import Dense, SimpleRNN, Input, LSTM, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
with open('train_data_true.txt', 'r', encoding='utf-8') as f:
    text_true = f.readlines()
    text_true[0] = text_true[0].replace(r'\ufeff', '')
    
with open('train_data_false', 'r', encoding='utf-8') as f:
    text_false = f.readlines()
    text_false[0] = text_false[0].replace(r'\ufeff', '')
    
texts = text_true + text_false
count_true = len(text_true)
count_false = len(text_false)
total_lines = count_true + count_false

In [ ]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)

tokenizer.fit_on_texts(texts)

In [ ]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])

In [ ]:
max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)
print( list(tokenizer.word_index.items()) )

In [ ]:
x = data_pad
y = np.array([[1, 0]] * count_true + [[0, 1]] * count_false)

print(x.shape, y.shape)

In [ ]:
indeces = np.random.choice(x.shape[0], size=x.shape[0], replace=False)
x = x[indeces]
y = y[indeces]

In [ ]:
model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length=max_text_len))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(2, activation='softmax'))


model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

In [ ]:
history = model.fit(x, y, batch_size=32, epochs=50)